# Definición parámetros

In [ ]:
# === OUT>%IN — Amount-OUT-30d: percentiles de la suma OUT Cash en 30 días =====
import pandas as pd
import numpy as np

pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# Parámetros
PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = ["I-1", "I-2"]       # <-- ajusta el sub-subsegmento
WINDOW_DAYS = 30
PCTS = [0.95, 0.97, 0.98, 0.99]
FILTER_TO_CASH = True
USE_ABS = True  # True recomendado para evitar signos contables

# Carga mínima
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df["tx_direction"].astype(str).str.title()
if FILTER_TO_CASH and "tx_type" in df.columns:
    df["tx_type"] = df["tx_type"].astype(str).str.title()

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))   

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

# Filtro: OUTBOUND (+ Cash opcional) y datos válidos
mask = (
    df["tx_direction"].eq("Outbound") &
    df["tx_date_time"].notna() &
    df["tx_base_amount"].notna()
)
if FILTER_TO_CASH and "tx_type" in df.columns:
    mask &= df["tx_type"].eq("Cash")

g = df.loc[mask, ["customer_id","tx_date_time","tx_base_amount"]].copy()
if g.empty:
    print("No hay transacciones elegibles para OUT>%IN (Amount-OUT-30d).")
else:
    g["amt"] = g["tx_base_amount"].abs() if USE_ABS else g["tx_base_amount"]

    parts = []
    for cid, sub in g.groupby("customer_id", sort=False):
        daily = sub.set_index("tx_date_time")["amt"].resample("D").sum()
        roll_sum = daily.rolling(f"{WINDOW_DAYS}D").sum()
        parts.append(roll_sum.rename(cid))

    if not parts:
        print("Sin series diarias para calcular la ventana.")
    else:
        s = pd.concat([ser.dropna().astype(float) for ser in parts], axis=0)
        q = s.quantile(PCTS) if len(s) else pd.Series(index=PCTS, dtype=float)

        out = pd.DataFrame({
            "percentil":    [f"p{int(p*100)}" for p in PCTS],
            "Amount_OUT_30d":[q.get(p, np.nan) for p in PCTS],
        })

        print(f"=== OUT>%IN — Percentiles de suma OUT {WINDOW_DAYS}d (CLP) ===")
        print(f"Clientes con OUT: {g['customer_id'].nunique():,} | Ventanas: {len(s):,}\n")
        display(out)


C:\Users\vinso\AppData\Local\Temp\ipykernel_49848\548647009.py:16: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== OUT>%IN — Percentiles de suma OUT 30d (CLP) ===
Clientes con OUT: 18 | Ventanas: 2,878



,percentil,Amount_OUT_30d
0,p95,"11,716,018,023"
1,p97,"12,242,640,000"
2,p98,"12,887,316,115"
3,p99,"13,552,737,912"


# Simulación alertas

In [3]:
# === OUT>%IN — Sensibilidad (Actual vs propuestos) =============================
# LÓGICA EXACTA (ventanas cliente–día):
#   sum(OUT) 30d > Amount  AND  OUT30 ∈ [Low%, High%] de IN30
# Notas: rolling "30D" incluye el día actual; usamos |tx_base_amount|.

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# ---- EDITA AQUÍ ---------------------------------------------------------------
PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = ["I-2"]              # <-- ajusta el sub-subsegmento
PARAMS = {
    "Actual": {"Amount_OUT_30d": 804_076_122, "Low": 125, "High": 200},
    "p95":    {"Amount_OUT_30d":   11_716_018_023, "Low": 125, "High": 200},
    "p97":    {"Amount_OUT_30d":   12_242_640_000, "Low": 125, "High": 200},
    "p99":    {"Amount_OUT_30d": 13_552_737_912, "Low": 125, "High": 200},
}
# -------------------------------------------------------------------------------

df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df["tx_direction"].astype(str).str.title()

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

base_mask = df["tx_date_time"].notna() & df["tx_base_amount"].notna() & df["customer_id"].notna()

OUT_ = df[base_mask & df["tx_direction"].eq("Outbound")][["customer_id","tx_date_time","tx_base_amount"]].copy()
IN_  = df[base_mask & df["tx_direction"].eq("Inbound")][["customer_id","tx_date_time","tx_base_amount"]].copy()

if OUT_.empty:
    print("No hay transacciones Outbound elegibles.")
else:
    parts=[]
    for cid, sub_out in OUT_.groupby("customer_id", sort=False):
        out_daily = (sub_out.set_index("tx_date_time")["tx_base_amount"]
                            .abs()
                            .resample("D")
                            .sum())
        sub_in = IN_[IN_["customer_id"].eq(cid)]
        if sub_in.empty:
            in_daily = pd.Series(0.0, index=out_daily.index)
        else:
            in_daily = (sub_in.set_index("tx_date_time")["tx_base_amount"]
                               .abs()
                               .resample("D")
                               .sum())

        start = min(out_daily.index.min(), in_daily.index.min())
        end   = max(out_daily.index.max(), in_daily.index.max())
        idx   = pd.date_range(start, end, freq="D")

        out_daily = out_daily.reindex(idx, fill_value=0.0)
        in_daily  = in_daily.reindex(idx,  fill_value=0.0)

        OUT30 = out_daily.rolling("30D", min_periods=1).sum()
        IN30  = in_daily.rolling("30D", min_periods=1).sum()

        parts.append(pd.DataFrame({"customer_id": cid, "date": idx, "OUT30": OUT30.values, "IN30": IN30.values}))

    M = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["customer_id","date","OUT30","IN30"])

    if M.empty:
        print("No se pudieron construir ventanas 30d (M vacío).")
    else:
        order = ["Actual","p85","p90","p95","p97","p98","p99"]
        if PARAMS:
            param_tbl = (pd.DataFrame(PARAMS).T
                           .loc[[k for k in order if k in PARAMS] or list(PARAMS.keys())]
                           .rename_axis("escenario").reset_index())
            print("=== OUT>%IN — Parámetros (Amount_OUT_30d, Low, High) ==="); display(param_tbl)

        counts = {}
        for k, v in PARAMS.items():
            A, L, H = float(v["Amount_OUT_30d"]), float(v["Low"]), float(v["High"])
            m = (
                (M["OUT30"] > A) &
                (M["IN30"] > 0) &
                (M["OUT30"] >= M["IN30"] * (L/100.0)) &
                (M["OUT30"] <= M["IN30"] * (H/100.0))
            )
            counts[k] = int(M.loc[m, ["customer_id","date"]].drop_duplicates().shape[0])

        if PARAMS:
            alerts = pd.DataFrame([{f"alertas_{name.lower()}": counts.get(name, 0)
                                    for name in ([k for k in order if k in PARAMS] or list(PARAMS.keys()))}])
            print("=== OUT>%IN — Alertas por escenario (ventanas cliente–día) ==="); display(alerts)


C:\Users\vinso\AppData\Local\Temp\ipykernel_24816\1830756130.py:20: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== OUT>%IN — Parámetros (Amount_OUT_30d, Low, High) ===


,escenario,Amount_OUT_30d,Low,High
0,Actual,804076122,125,200
1,p95,11716018023,125,200
2,p97,12242640000,125,200
3,p99,13552737912,125,200


=== OUT>%IN — Alertas por escenario (ventanas cliente–día) ===


,alertas_actual,alertas_p95,alertas_p97,alertas_p99
0,171,8,2,1
